In [1]:
import numpy as np
from scipy.io import loadmat
from scipy.optimize import minimize
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

def preprocess():
    """ 
     Input:
     Although this function doesn't have any input, you are required to load
     the MNIST data set from file 'mnist_all.mat'.

     Output:
     train_data: matrix of training set. Each row of train_data contains 
       feature vector of a image
     train_label: vector of label corresponding to each image in the training
       set
     validation_data: matrix of training set. Each row of validation_data 
       contains feature vector of a image
     validation_label: vector of label corresponding to each image in the 
       training set
     test_data: matrix of training set. Each row of test_data contains 
       feature vector of a image
     test_label: vector of label corresponding to each image in the testing
       set
    """

    mat = loadmat('mnist_all.mat')  # loads the MAT object as a Dictionary

    n_feature = mat.get("train1").shape[1]
    n_sample = 0
    for i in range(10):
        n_sample = n_sample + mat.get("train" + str(i)).shape[0]
    n_validation = 1000
    n_train = n_sample - 10 * n_validation

    # Construct validation data
    validation_data = np.zeros((10 * n_validation, n_feature))
    for i in range(10):
        validation_data[i * n_validation:(i + 1) * n_validation, :] = mat.get("train" + str(i))[0:n_validation, :]

    # Construct validation label
    validation_label = np.ones((10 * n_validation, 1))
    for i in range(10):
        validation_label[i * n_validation:(i + 1) * n_validation, :] = i * np.ones((n_validation, 1))

    # Construct training data and label
    train_data = np.zeros((n_train, n_feature))
    train_label = np.zeros((n_train, 1))
    temp = 0
    for i in range(10):
        size_i = mat.get("train" + str(i)).shape[0]
        train_data[temp:temp + size_i - n_validation, :] = mat.get("train" + str(i))[n_validation:size_i, :]
        train_label[temp:temp + size_i - n_validation, :] = i * np.ones((size_i - n_validation, 1))
        temp = temp + size_i - n_validation

    # Construct test data and label
    n_test = 0
    for i in range(10):
        n_test = n_test + mat.get("test" + str(i)).shape[0]
    test_data = np.zeros((n_test, n_feature))
    test_label = np.zeros((n_test, 1))
    temp = 0
    for i in range(10):
        size_i = mat.get("test" + str(i)).shape[0]
        test_data[temp:temp + size_i, :] = mat.get("test" + str(i))
        test_label[temp:temp + size_i, :] = i * np.ones((size_i, 1))
        temp = temp + size_i

    # Delete features which don't provide any useful information for classifiers
    sigma = np.std(train_data, axis=0)
    index = np.array([])
    for i in range(n_feature):
        if (sigma[i] > 0.001):
            index = np.append(index, [i])
    train_data = train_data[:, index.astype(int)]
    validation_data = validation_data[:, index.astype(int)]
    test_data = test_data[:, index.astype(int)]

    # Scale data to 0 and 1
    train_data /= 255.0
    validation_data /= 255.0
    test_data /= 255.0

    return train_data, train_label, validation_data, validation_label, test_data, test_label


def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))


def blrObjFunction(initialWeights, *args):
    """
    blrObjFunction computes 2-class Logistic Regression error function and
    its gradient.

    Input:
        initialWeights: the weight vector (w_k) of size (D + 1) x 1 
        train_data: the data matrix of size N x D
        labeli: the label vector (y_k) of size N x 1 where each entry can be either 0 or 1 representing the label of corresponding feature vector

    Output: 
        error: the scalar value of error function of 2-class logistic regression
        error_grad: the vector of size (D+1) x 1 representing the gradient of
                    error function
    """
    train_data, labeli = args

    n_data = train_data.shape[0]
    n_features = train_data.shape[1]
    error = 0
    error_grad = np.zeros((n_features + 1, 1))

    ##################
    # YOUR CODE HERE #
    ##################
    # HINT: Do not forget to add the bias term to your input data
    #add bias item for input data
    X0 = np.ones((n_data, 1)) #N x 1
#     print('shape of X0: ', X0.shape)
    new_train_data = np.hstack((X0, train_data)) #N x D+1
    yn = labeli #N x 1
    #compute theta
    z = np.dot(new_train_data, initialWeights) #N x 1
    theta = sigmoid(z) #let theta value in (0,1)   #(N, ) 
    theta = theta.reshape(theta.shape[0], 1) #(N,1)
    #compute error
    error = -(1 / n_data) * (np.sum((yn * np.log(theta)) + (1 - yn) * np.log(1 - theta))) #we don't need to compute likelihood.
    #compute gradient error
    error_grad = (1 / n_data) * (np.sum(((theta - yn) * new_train_data), axis=0)).T #(N,1)*(N,D+1)=(N,D+1) 

    return error, error_grad


def blrPredict(W, data):
    """
     blrObjFunction predicts the label of data given the data and parameter W 
     of Logistic Regression
     
     Input:
         W: the matrix of weight of size (D + 1) x 10. Each column is the weight 
         vector of a Logistic Regression classifier.
         X: the data matrix of size N x D
         
     Output: 
         label: vector of size N x 1 representing the predicted label of 
         corresponding feature vector given in data matrix

    """
    label = np.zeros((data.shape[0], 1))

    ##################
    # YOUR CODE HERE #
    ##################
    # HINT: Do not forget to add the bias term to your input data
    #add bias term for input data
    X0 = np.ones((data.shape[0], 1))
    input_data = np.hstack((X0, data))
    #compute predicted label
    z = np.dot(input_data,W) # w = [w0,w1,...,wD], data = [1,x1,x2,...,xD]
    pred = sigmoid(z)  #(N,10) ->choose the max probability of entry in each row, each entry probability is in (0,1).
    label_maxindex = np.argmax(pred, axis = 1)  #N
    
    label = np.reshape(label_maxindex, (data.shape[0], 1))
    return label


def mlrObjFunction(params, *args):
    """
    mlrObjFunction computes multi-class Logistic Regression error function and
    its gradient.

    Input:
        initialWeights_b: the weight vector of size (D + 1) x 10
        train_data: the data matrix of size N x D
        labeli: the label vector of size N x 1 where each entry can be either 0 or 1
                representing the label of corresponding feature vector

    Output:
        error: the scalar value of error function of multi-class logistic regression
        error_grad: the vector of size (D+1) x 10 representing the gradient of
                    error function
    """
    train_data, labeli = args
    
    n_data = train_data.shape[0]
    n_feature = train_data.shape[1]
    error = 0
    error_grad = np.zeros((n_feature + 1, n_class))
    
    ##################
    # YOUR CODE HERE #
    ##################
    # HINT: Do not forget to add the bias term to your input data
    #add bias item for input data
    X0 = np.ones((n_data, 1))
    new_train_data = np.hstack((X0, train_data)) #(N, D+1)
    #compute wk
    wk = params.reshape((n_feature + 1, n_class)) #(D+1, 10)
    #compute posterior probabilities
    vector = np.dot(new_train_data, wk) #(N, 10)
    post_prob = np.exp(vector) / np.reshape(np.sum(np.exp(vector), axis=1), (n_data, 1))   #(N, 10)
#     print((labeli * np.log(post_prob)).shape) #(N, 1)*(N, 10) = (N, 10)
    #compute error
    error = -np.sum(labeli * np.log(post_prob))#(N, 10).sum -> total error in only one scala value.
########### compute total error of each category
    train_error_mlr = []
    for i in range(n_class):
        error_mlr = -np.sum(labeli[:, i] * np.log(post_prob[:,i]))
        train_error_mlr.append(error_mlr)
###########          
    #compute gradient error
    for i in range(n_class):
        error_grad[:, i] = np.sum((post_prob[:, i] - np.transpose(labeli[:, i])) * np.transpose(new_train_data), axis=1)
    error_grad = error_grad.flatten()

    return error, error_grad, train_error_mlr


def mlrPredict(W, data):
    """
     mlrObjFunction predicts the label of data given the data and parameter W
     of Logistic Regression

     Input:
         W: the matrix of weight of size (D + 1) x 10. Each column is the weight
         vector of a Logistic Regression classifier.
         X: the data matrix of size N x D

     Output:
         label: vector of size N x 1 representing the predicted label of
         corresponding feature vector given in data matrix

    """
    label = np.zeros((data.shape[0], 1))

    ##################
    # YOUR CODE HERE #
    ##################
    # HINT: Do not forget to add the bias term to your input data
    #add bias item for input data
    X0 = np.ones((data.shape[0], 1))
    input_data = np.hstack((X0, data))
    #compute predicted label
    z = np.dot(input_data, W)
    pred = np.exp(z) / np.reshape(np.sum(np.exp(z), axis = 1), (data.shape[0], 1)) #(N,10) ->choose the max one of each row
    label_maxindex = np.argmax(pred, axis = 1)  #N
    label = np.reshape(label_maxindex, (data.shape[0], 1))
    
    return label


"""
Script for Logistic Regression
"""
train_data, train_label, validation_data, validation_label, test_data, test_label = preprocess()

n_class = 10 # number of classes
n_train = train_data.shape[0] # number of classes
n_feature = train_data.shape[1] # number of features

######compute total error of training data for each category
Y = np.zeros((n_train, n_class))
for i in range(n_class):
    Y[:, i] = (train_label == i).astype(int).ravel()

# Logistic Regression with Gradient Descent
W = np.zeros((n_feature + 1, n_class))
initialWeights = np.zeros((n_feature + 1, 1))
opts = {'maxiter': 100}
for i in range(n_class):
    labeli = Y[:, i].reshape(n_train, 1)
    args = (train_data, labeli)
    nn_params = minimize(blrObjFunction, initialWeights, jac=True, args=args, method='CG', options=opts)
    W[:, i] = nn_params.x.reshape((n_feature + 1,))
    #compute total error of training data with respect to each category
    error, error_grad = blrObjFunction(W[:, i], *args)
    print('\n train_error_blr: ', error)
print('-'*100)


 train_error_blr:  0.02062556400652329

 train_error_blr:  0.02071294577137169

 train_error_blr:  0.06280477163815648

 train_error_blr:  0.0755166637301917

 train_error_blr:  0.0442572674594386

 train_error_blr:  0.08168637840353371

 train_error_blr:  0.03454490617473333

 train_error_blr:  0.043831522256309305

 train_error_blr:  0.10996929922813548

 train_error_blr:  0.09686647290809093
----------------------------------------------------------------------------------------------------


### training error in mlr

In [2]:
# FOR EXTRA CREDIT ONLY
n_class = 10 # number of classes
n_train = train_data.shape[0] # number of classes
n_feature = train_data.shape[1] # number of features
######compute total error of training data for each category
Y = np.zeros((n_train, n_class))
for i in range(n_class):
    Y[:, i] = (train_label == i).astype(int).ravel()

W_b = np.zeros((n_feature + 1, n_class))
initialWeights_b = np.zeros((n_feature + 1, n_class))
opts_b = {'maxiter': 100}

args_b = (train_data, Y)
nn_params = minimize(mlrObjFunction, initialWeights_b, jac=True, args=args_b, method='CG', options=opts_b)
W_b = nn_params.x.reshape((n_feature + 1, n_class))

error_mlr, error_grad_mlr, train_error_mlr = mlrObjFunction(W_b, *args_b)
print('\n train_error_mlr: ', train_error_mlr)


 train_error_mlr:  [503.77379245715895, 627.6773122382641, 1640.814965978841, 1600.404478485907, 1115.3849518299317, 1758.2481442012422, 698.7854563388089, 1112.9571035254132, 1745.727377769515, 1572.0684136988375]


# Prediction Accuracy

### test data in blr

In [3]:
# Find the accuracy on Training Dataset
predicted_label = blrPredict(W, train_data)
print('\n Training set Accuracy:' + str(100 * np.mean((predicted_label == train_label).astype(float))) + '%')

# Find the accuracy on Validation Dataset
predicted_label = blrPredict(W, validation_data)
print('\n Validation set Accuracy:' + str(100 * np.mean((predicted_label == validation_label).astype(float))) + '%')

# Find the accuracy on Testing Dataset
predicted_label = blrPredict(W, test_data)
print('\n Testing set Accuracy:' + str(100 * np.mean((predicted_label == test_label).astype(float))) + '%')


 Training set Accuracy:92.742%

 Validation set Accuracy:91.53999999999999%

 Testing set Accuracy:92.01%


### test data in mlr

In [4]:
# Find the accuracy on Training Dataset
predicted_label_b = mlrPredict(W_b, train_data)
print('\n Training set Accuracy:' + str(100 * np.mean((predicted_label_b == train_label).astype(float))) + '%')

# Find the accuracy on Validation Dataset
predicted_label_b = mlrPredict(W_b, validation_data)
print('\n Validation set Accuracy:' + str(100 * np.mean((predicted_label_b == validation_label).astype(float))) + '%')

# Find the accuracy on Testing Dataset
predicted_label_b = mlrPredict(W_b, test_data)
print('\n Testing set Accuracy:' + str(100 * np.mean((predicted_label_b == test_label).astype(float))) + '%')


 Training set Accuracy:93.112%

 Validation set Accuracy:92.39%

 Testing set Accuracy:92.54%


# Test error

### test error in blr

In [5]:
n_class = 10 # number of features
n_test = test_data.shape[0] # number of features
n_feature = test_data.shape[1] # number of features

######compute total error of training data for each category
Y = np.zeros((n_test, n_class))
for i in range(n_class):
    Y[:, i] = (test_label == i).astype(int).ravel()

# Logistic Regression with Gradient Descent
initialWeights = np.zeros((n_feature + 1, 1))
opts = {'maxiter': 100}
for i in range(n_class):
    labeli_test = Y[:, i].reshape(n_test, 1)
    args_test = (test_data, labeli_test)
    #compute total error of training data with respect to each category
    error_test, error_grad = blrObjFunction(W[:, i], *args_test)
    print('\n test_error_blr: ', error_test)
print('-'*100)


 test_error_blr:  0.025969725707206567

 test_error_blr:  0.02290759144938041

 test_error_blr:  0.07267805840620196

 test_error_blr:  0.07151132476989247

 test_error_blr:  0.051102537456837106

 test_error_blr:  0.08464830954311611

 test_error_blr:  0.03775557630140572

 test_error_blr:  0.053979648189005344

 test_error_blr:  0.11276250613315576

 test_error_blr:  0.10217055554644983
----------------------------------------------------------------------------------------------------


### test error in mlr

In [6]:
# FOR EXTRA CREDIT ONLY
initialWeights_b = np.zeros((n_feature + 1, n_class))
opts_b = {'maxiter': 100}
Y = np.zeros((n_test, n_class))
for i in range(n_class):
    Y[:, i] = (test_label == i).astype(int).ravel()
    
args_b = (test_data, Y)
error_mlr, error_grad_mlr, test_error_mlr = mlrObjFunction(W_b, *args_b)
print('\n test_error_mlr: ', test_error_mlr)


 test_error_mlr:  [69.18157290263284, 118.78492930024363, 400.5346028064311, 272.59705148931, 221.2825040256135, 383.3118682762621, 180.7790320284797, 300.989824923036, 391.7513910352269, 336.5870310371406]


# SVM

In [7]:
"""
Script for Support Vector Machine
"""

print('\n\n--------------SVM-------------------\n\n')
##################
# YOUR CODE HERE #
##################

#choose 10000 rows randomly of dataset
index1 = np.random.choice(train_data.shape[0], int(0.2*train_data.shape[0]), replace=False)  
X_train_data = train_data[index1]
y_train_data = train_label[index1]
index2 = np.random.choice(validation_data.shape[0], int(0.2*validation_data.shape[0]), replace=False)  
X_validation_Data = validation_data[index2]
y_validation_Data = validation_label[index2]
index3 = np.random.choice(test_data.shape[0], int(0.2*test_data.shape[0]), replace=False)  
X_test_data = test_data[index3]
y_test_data = test_label[index3]
######linear kernel
#fitting SVM into training, validation, test data
##training data
classifier1 = SVC(kernel='linear', random_state = 501)
classifier1.fit(X_train_data, y_train_data) 
y_pred1 = classifier1.predict(X_train_data)
acc1 = accuracy_score(y_pred1, y_train_data)*100
print('\n train_accuracy of SVM using linear kernel: ' + str(acc1) + '%')
##validation data
y_pred2 = classifier1.predict(X_validation_Data)
acc2 = accuracy_score(y_pred2, y_validation_Data)*100
print(' validation_accuracy of SVM using linear kernel: ' + str(acc2) + '%')
##test data
y_pred3 = classifier1.predict(X_test_data)
acc3 = accuracy_score(y_pred3, y_test_data)*100
print(' test_accuracy of SVM using linear kernel: ' + str(acc3) + '%')

######rbf with gamma=1
#fitting SVM into training, validation, test data
##training data
classifier2 = SVC(kernel='rbf', gamma=1, random_state = 501)
classifier2.fit(X_train_data, y_train_data) 
y_pred11 = classifier2.predict(X_train_data)
acc11 = accuracy_score(y_pred11, y_train_data)*100
print('\n train_accuracy of SVM using rbf and gamma=1 kernel: ' + str(acc11) + '%')
##validation data
y_pred22 = classifier2.predict(X_validation_Data)
acc22 = accuracy_score(y_pred22, y_validation_Data)*100
print(' validation_accuracy of SVM using rbf and gamma=1 kernel: ' + str(acc22) + '%')
##test data 
y_pred33 = classifier2.predict(X_test_data)
acc33 = accuracy_score(y_pred33, y_test_data)*100
print(' test_accuracy of SVM using rbf and gamma=1 kernel: ' + str(acc33) + '%')

######rbf with gamma=default
#fitting SVM into training, validation, test data
##training data
classifier3 = SVC(kernel='rbf', gamma='auto', random_state = 501)
classifier3.fit(X_train_data, y_train_data) 
y_pred111 = classifier3.predict(X_train_data)
acc111 = accuracy_score(y_pred111, y_train_data)*100
print('\n train_accuracy of SVM using rbf and gamma=auto kernel: ' + str(acc111) + '%')
##validation data
y_pred222 = classifier3.predict(X_validation_Data)
acc222 = accuracy_score(y_pred222, y_validation_Data)*100
print(' validation_accuracy of SVM using rbf and gamma=auto kernel: ' + str(acc222) + '%')
##test data
y_pred333 = classifier3.predict(X_test_data)
acc333 = accuracy_score(y_pred333, y_test_data)*100
print(' test_accuracy of SVM using rbf and gamma=auto kernel: ' + str(acc333) + '%')

######rbf with gamma=default,C = 1,10,20,30,...,100 and plot graph of accuracy
C = [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
for i in C:
    ##training data
    classifier = SVC(kernel='rbf', gamma='auto', C=i, random_state = 501)
    classifier.fit(X_train_data, y_train_data) 
    y_pred1111 = classifier.predict(X_train_data)
    acc_c1 = accuracy_score(y_pred1111, y_train_data)*100
    print('\n train_accuracy of SVM using rbf, gamma=auto and C=' + str(i) + ' kernel: ' + str(acc_c1) + '%')
    ##validation data
    y_pred2222 = classifier.predict(X_validation_Data)
    acc_c2 = accuracy_score(y_pred2222, y_validation_Data)*100
    print(' validation_accuracy of SVM using rbf, gamma=auto and C=' + str(i) + ' kernel: ' + str(acc_c2) + '%')
    ##test data
    y_pred3333 = classifier.predict(X_test_data)
    acc_c3 = accuracy_score(y_pred3333, y_test_data)*100
    print(' test_accuracy of SVM using rbf, gamma=auto and C=' + str(i) + ' kernel: ' + str(acc_c3) + '%')




--------------SVM-------------------




/Users/maweibin/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



 train_accuracy of SVM using linear kernel: 99.64%
 validation_accuracy of SVM using linear kernel: 91.4%
 test_accuracy of SVM using linear kernel: 92.0%

 train_accuracy of SVM using rbf and gamma=1 kernel: 100.0%
 validation_accuracy of SVM using rbf and gamma=1 kernel: 10.2%
 test_accuracy of SVM using rbf and gamma=1 kernel: 11.700000000000001%

 train_accuracy of SVM using rbf and gamma=auto kernel: 92.29%
 validation_accuracy of SVM using rbf and gamma=auto kernel: 92.0%
 test_accuracy of SVM using rbf and gamma=auto kernel: 92.9%

 train_accuracy of SVM using rbf, gamma=auto and C=1 kernel: 92.29%
 validation_accuracy of SVM using rbf, gamma=auto and C=1 kernel: 92.0%
 test_accuracy of SVM using rbf, gamma=auto and C=1 kernel: 92.9%

 train_accuracy of SVM using rbf, gamma=auto and C=10 kernel: 96.34%
 validation_accuracy of SVM using rbf, gamma=auto and C=10 kernel: 94.39999999999999%
 test_accuracy of SVM using rbf, gamma=auto and C=10 kernel: 94.89999999999999%

 train_accu